## Pointing error CORALIE analysis using plate solving

In [1]:
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord, AltAz, EarthLocation
from astropy.time import Time
import os
import subprocess
import glob
#from astroquery.astrometry_net import AstrometryNet
from astropy.io import fits
import sys
from termcolor import colored

### Platesolving the images

In [3]:
# Input and output directories
#fits_dir = "/home/alberte2/Sample_docs/Pm_Coralie_2024-12-11/"
#output_dir = "/home/alberte2/Sample_docs/Pm_Coralie_2024-12-11/solved/"
fits_dir = "/home/alberte2/Sample_docs/Archives_CORALIE/"
output_dir = "/home/alberte2/Sample_docs/Archives_CORALIE/solved/"
os.makedirs(output_dir, exist_ok=True)

# Pixel scale range (arcsec per pixel)
scale_low = 0.24
scale_high = 0.28  # True value is 0.2575

# Get a list of FITS files in the input directory
fits_files = glob.glob(os.path.join(fits_dir, "*.fits"))

# Loop through the FITS files
for fits_path in fits_files:
    file_name = os.path.basename(fits_path)
    output_name = file_name.replace(".fits", ".solved.fits")

    # Open FITS file
    with fits.open(fits_path) as hdul:
        header = hdul[0].header
        ra_approx = header['HIERARCH ESO TEL TARG ALPHA INST']  # Fetch RA from header
        dec_approx = header['HIERARCH ESO TEL TARG DELTA INST']  # Fetch Dec from header

        try:
            # Run solve-field for each FITS file with specified parameters
            result = subprocess.run(
                [
                    "ssh", "glsastro",
                    "cd /home/alberte2/Sample_docs/Archives_CORALIE/ &&", 
                    #"cd /home/alberte2/Sample_docs/Pm_Coralie_2024-12-11/solved/ &&",  # Ensure working directory
                    "solve-field",
                    #"--verbose",  # Enables verbose output (chatty)
                    "--overwrite",  # Overwrite existing output files
                    "--no-verify",  # Skips verification of FITS headers
                    #"--no-fits2fits",  # Avoid creating intermediate FITS files
                    f"--ra {ra_approx}",  # Approximate RA (Right Ascension)
                    f"--dec {dec_approx}",  # Approximate Dec (Declination)
                    "--scale-units", "arcsecperpix",  # Scale units
                    f"--scale-low {scale_low}",  # Minimum pixel scale
                    f"--scale-high {scale_high}",  # Maximum pixel scale
                    "--radius", "3",  # Search radius
                    "--depth", "50", #controls number of features to be considered during solving. Higher values increases accuracy but takes longer time.
                    "--objs", "100", #maximum number of objects used for solving
                    "-z 4",
                    "--no-plots",  # Suppress plot outputs
                    f"--dir {output_dir}",  # Directory for output files
                    f"--new-fits {output_name}",  # Name of the new FITS file
                    fits_path  # The FITS file to be solved
                ],
                check=True,  # Raise an exception if the command fails
                stdout=subprocess.PIPE,  # Capture stdout
                stderr=subprocess.PIPE,  # Capture stderr
                text=True  # Ensure text output
            )
            # Display output
            print(colored("*******************************************", "green"))
            print(" ")
            print(colored(f"Output for {file_name}:", "blue", attrs=["bold"]))
            print(result.stdout)  # Show standard output
            print(colored("*******************************************", "green"))
            print(colored(f"SOLVED: {file_name}", "green", attrs=["bold"]))

        except subprocess.CalledProcessError as e:
            print(colored("*******************************************", "red"))
            print(colored(f"Failed to solve: {file_name}.", "red", attrs=["bold"]))
            print(f"Error: {e.stderr}")
            print(colored("*******************************************", "red"))
            print(" ")


*******************************************
 
Output for PCORALIE.2025-01-04T08:48:11.000.fits:
Reading input file 1 of 1: "/home/alberte2/Sample_docs/Archives_CORALIE/PCORALIE.2025-01-04T08:48:11.000.fits"...
Extracting sources...
Downsampling by 4...
simplexy: found 10 sources.
Solving...
Failed to add index "/usr/share/astrometry/index-tycho2-19.bigendian.fits".
Failed to add index "/usr/share/astrometry/index-tycho2-18.bigendian.fits".
Failed to add index "/usr/share/astrometry/index-tycho2-17.bigendian.fits".
Failed to add index "/usr/share/astrometry/index-tycho2-16.bigendian.fits".
Failed to add index "/usr/share/astrometry/index-tycho2-15.bigendian.fits".
Failed to add index "/usr/share/astrometry/index-tycho2-14.bigendian.fits".
Failed to add index "/usr/share/astrometry/index-tycho2-13.bigendian.fits".
Failed to add index "/usr/share/astrometry/index-tycho2-12.bigendian.fits".
Failed to add index "/usr/share/astrometry/index-tycho2-11.bigendian.fits".
Failed to add index "/us

In [4]:
# Set up Astrometry.net client
ast = AstrometryNet()
ast.api_key = 'assceldzujfiebeh'  # Replace with your API key

# Input and output directories
fits_dir = "/home/alberte2/Sample_docs/Pm_Coralie_2024-12-11/"
output_dir = "/home/alberte2/Sample_docs/Pm_Coralie_2024-12-11/solved/"
os.makedirs(output_dir, exist_ok=True)

# Get a list of FITS files in the input directory
fits_files = glob.glob(os.path.join(fits_dir, "*.fits"))

# Loop through the FITS files
for fits_path in fits_files:
    file_name = os.path.basename(fits_path)
    output_file = os.path.join(output_dir, file_name.replace(".fits", "_solved.fits"))

    # Skip if already solved
    if os.path.exists(output_file):
        print(f"File {output_file} already solved. Skipping...")
        continue

    # Open FITS file to check headers (optional)
    try:
        with fits.open(fits_path) as hdul:
            #header = hdul[0].header
            print(f"Processing file: {file_name}")
            #print("Header information (optional):")
            #print(header)
    except Exception as e:
        print(f"Error reading FITS file {file_name}: {e}")
        continue

    # Upload FITS file for plate-solving
    try:
        print(f"Uploading {file_name} to Astrometry.net...")
        submission_id = ast.submit_from_path(fits_path)
        print("Submission ID:", submission_id)

        # Wait for the solution
        result = ast.monitor_submission(submission_id, solve_timeout=300)
        print(f"Plate-solving complete for {file_name}!")

        # Save the solved file
        if 'wcs_file' in result:
            wcs_file = result['wcs_file']
            with open(output_file, 'wb') as f:
                f.write(wcs_file)
            print(f"Solved file saved to: {output_file}")
        else:
            print(f"No WCS file returned for {file_name}.")
    except Exception as e:
        print(f"Plate-solving failed for {file_name}: {e}")


Processing file: PCoralie0_HD65277.fits
Uploading PCoralie0_HD65277.fits to Astrometry.net...
Plate-solving failed for PCoralie0_HD65277.fits: 'AstrometryNetClass' object has no attribute 'submit_from_path'
Processing file: PCoralie1_HD78663.fits
Uploading PCoralie1_HD78663.fits to Astrometry.net...
Plate-solving failed for PCoralie1_HD78663.fits: 'AstrometryNetClass' object has no attribute 'submit_from_path'
Processing file: PCoralie2_HD65243.fits
Uploading PCoralie2_HD65243.fits to Astrometry.net...
Plate-solving failed for PCoralie2_HD65243.fits: 'AstrometryNetClass' object has no attribute 'submit_from_path'
Processing file: PCoralie3_HD76849.fits
Uploading PCoralie3_HD76849.fits to Astrometry.net...
Plate-solving failed for PCoralie3_HD76849.fits: 'AstrometryNetClass' object has no attribute 'submit_from_path'
Processing file: PCoralie4_HD82943.fits
Uploading PCoralie4_HD82943.fits to Astrometry.net...
Plate-solving failed for PCoralie4_HD82943.fits: 'AstrometryNetClass' object h

### Process solved files

In [7]:
# Observatory location for CORALIE (example: La Silla)
obs_location = EarthLocation(lon=-70.732951945082, lat=-29.259497822801, height=2378.0)

# Step 2: Process FITS Files
def process_fits_images(output_dir):
    pointing_data = []

    for file_name in os.listdir(output_dir):
        if file_name.endswith('.solved.fits'):
            fits_path = os.path.join(output_dir, file_name)

            try:
                # Open FITS file
                with fits.open(fits_path) as hdul:
                    header = hdul[0].header
                    w = WCS(header)
                    #print('wcs: ', w) 
                    obs_time = Time(float(header.get('HIERARCH ESO TEL TARG TUNIX INST')), format = 'unix')

                    #print("Obs time: ", obs_time)

                    # Extract expected azimuth and elevation
                    az_expected = header.get('HIERARCH ESO TEL TARG AZI INST', None) % 360
                    el_expected = header.get('HIERARCH ESO TEL TARG ELE INST', None)

                    # Fiber pixel coordinates
                    fiber_x = header.get('HIERARCH FIBER XREFCUR', None)
                    fiber_y = header.get('HIERARCH FIBER YREFCUR', None)

                    # Check for missing data
                    if az_expected is None or el_expected is None:
                        print(f"Missing AZIMUTH/ELEVATIO in {file_name}. Skipping...")
                        continue
                    if fiber_x is None or fiber_y is None:
                        print(f"Missing FIBER_X/FIBER_Y in {file_name}. Skipping...")
                        continue

                    # Convert fiber coordinates to RA/Dec using WCS
                    fiber_world_coords = w.pixel_to_world(fiber_x, fiber_y)
                    #print(fiber_world_coords)
                    # Transform RA/Dec to Azimuth/Elevation
                    sky_coord = SkyCoord(
                        ra=fiber_world_coords.ra, dec=fiber_world_coords.dec,
                        frame="icrs", unit="deg"
                    )
                    altaz_frame = AltAz(location=obs_location, obstime=obs_time)
                    altaz_coord = sky_coord.transform_to(altaz_frame)

                    az_actual = (altaz_coord.az.deg + 180) % 360
                    el_actual = altaz_coord.alt.deg
                    #print("az actual: ", az_actual)

                    # Save data
                    pointing_data.append({
                        #'file': file_name,
                        'az_expected': az_expected,
                        'el_expected': el_expected,
                        'az_actual': az_actual,
                        'el_actual': el_actual,
                    })
            except Exception as e:
                print(f"Error processing {file_name}: {e}")

    return pointing_data


In [9]:
def create_pointing_data_file(data, output_file):
    try:
        with open(output_file, "w+") as out:
            out.write("! Pointing Data for CORALIE\n")
            out.write(":NODA\n")
            out.write("-29 15 24.0\n\n")  # Latitude

            for entry in data:
                out.write(f"{entry['az_expected']} {entry['el_expected']} {entry['az_actual']} {entry['el_actual']}\n")
            
            out.write("END\n")
        print(f"Pointing data file created at {output_file}")
    except Exception as e:
        print(f"Error creating pointing data file: {e}")
#for trouble shooting. not normally incuded in the script
output_file = "/home/alberte2/Sample_docs/Archives_CORALIE/test_cor.dat"  # Replace with desired output file
#output_dir = "/home/alberte2/Sample_docs/Pm_Coralie_2024-12-11/"  # Replace with desired TPoint output directory

# Process data
pointing_data = process_fits_images(output_dir)
# Prepare data file for TPoint analysis (no corrections applied)
create_pointing_data_file(pointing_data, output_file)

Pointing data file created at /home/alberte2/Sample_docs/Archives_CORALIE/test_cor.dat


In [12]:
# Step 3: Generate TPoint-Compatible Data File
def create_pointing_data_file(data, output_file):
    try:
        with open(output_file, "w+") as out:
            out.write("! Pointing Data for CORALIE\n")
            out.write(":NODA\n")
            out.write("-29 15 24.0\n\n")  # Latitude

            for entry in data:
                out.write(f"{entry['az_expected']} {entry['el_expected']} {entry['az_actual']} {entry['el_actual']}\n")
            
            out.write("END\n")
        print(f"Pointing data file created at {output_file}")
    except Exception as e:
        print(f"Error creating pointing data file: {e}")


# Step 4: Compute Pointing Model
def compute_pointing_model(data_file, output_dir, model_name):
    os.makedirs(output_dir, exist_ok=True)
    tpoint_script = os.path.join(output_dir, "TPoint.dat")
    
    try:
        with open(tpoint_script, "w+") as f:
            f.write(f"indat {data_file}\n")
            f.write("USE AN AW IA IE CA TF PZZ3 PZZ5 HZCZ4\n")
            f.write("fit\n")
            f.write("call a9\n")  # creates 9 auto plots
            f.write("spawn sleep 8\n")  # Adds a delay for plotting
            f.write(f"OUTMOD {model_name}.tp\n")
            f.write(f"GC P {model_name}.ps\n")
            f.write("end\n")

        os.chdir(output_dir)
        subprocess.run(f"tpoint < {tpoint_script}", shell=True, check=True)
        print(colored("Pointing model computed successfully.", "green"))
    except Exception as e:
        print(colored(f"Error computing pointing model: {e}", "red"))


In [13]:
# Example Execution
#fits_dir = "/home/alberte2/Sample_docs/Pm_Coralie_2024-12-11/"  # Replace with actual path
output_file = "/home/alberte2/Sample_docs/Pm_Coralie_2024-12-11/coralie_astrometry_pm-17-01-2025.dat"  # Replace with desired output file
#output_dir = "/home/alberte2/Sample_docs/Pm_Coralie_2024-12-11/"  # Replace with desired TPoint output directory

# Process data
pointing_data = process_fits_images(output_dir)

# Prepare data file for TPoint analysis (no corrections applied)
create_pointing_data_file(pointing_data, output_file)

# Compute pointing model (if needed for analysis)
compute_pointing_model(output_file,fits_dir, "coralie_pointing_model-17-01-2025")

Pointing data file created at /home/alberte2/Sample_docs/Pm_Coralie_2024-12-11/coralie_astrometry_pm-17-01-2025.dat
running /opt/t4/beta/src/simond/tpoint/bin/tpt / / /dev/null/ /opt/t4/beta/src/simond/tpoint/etc/tpoint/tpoint.ini
    initialization file: /opt/t4/beta/src/simond/tpoint/etc/tpoint/tpoint.ini
    log file: /dev/null/













+ - - - - - - - - - - - - - - - - - - - - +
|                 TPOINT                  |
|   Telescope Pointing Analysis System    |
|              Version 5.2-1              |
+ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ +

There are 54 standard pointing terms.
Reading procedures from file /opt/t4/beta/src/simond/tpoint/etc/tpoint/procs.dat ...
244 lines input.
Reading star catalogue entries from file /opt/t4/beta/src/simond/tpoint/etc/tpoint/stars.dat ...
209 stars input.

TPOINT ready for use:  type HELP for assistance, END to quit.

* ! Pointing Data for CORALIE
:NODA
-29 15 24.0

205.59162 59.10984 205.59024436012479 59.033174212182956
230.8996

### Other important things to include in the above script are plots of the pointing error in az and ele vs date, the pointing infomation as I did for the other ones, polar plots of the grid and possibly other important points.